In [1]:
%load_ext autoreload
%autoreload 2

# Exp 1: Reemplazo de datos missing
Primero quiero testear quien agarra con mas accuracy, hands + pose o hollistic

In [2]:
from src.mediapipe.render import render_frame, render_holistic_frame
from concurrent.futures import ThreadPoolExecutor
from src.utils.video import frame_reader
import mediapipe as mp
import cv2

In [5]:
cap_rh = cv2.VideoCapture("../data/LSA64/video/001_001_001.mp4")
cap_bh = cv2.VideoCapture("../data/LSA64/video/031_001_001.mp4")

In [7]:
# Prueba de render
with mp.solutions.pose.Pose() as pose, mp.solutions.hands.Hands(max_num_hands=2) as hands, \
     ThreadPoolExecutor(max_workers=2) as ex:

    for frame in frame_reader(cap_bh, fps=12):
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        fut_pose  = ex.submit(pose.process, rgb)
        fut_hands = ex.submit(hands.process, rgb)

        pose_results  = fut_pose.result()
        hands_results = fut_hands.result()

        render_frame(frame, pose_results, hands_results)

cv2.destroyAllWindows()

TypeError: 'type' object is not iterable

In [10]:
# Prueba de render usando holistic, a ver si captura mejor la info de las manos
with mp.solutions.holistic.Holistic() as holistic:
    for frame in frame_reader(cap_bh, fps=12):
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        holistic_res = holistic.process(rgb)
        render_holistic_frame(frame, holistic_res)

cv2.destroyAllWindows()

# Resultados
Parece que holistic captura todos los landmarks con mayor confianza que hands + pose, pero tengo que probar un poco mas. Por ahora me voy a quedar con holistic porque es la solucion mas facil y siento que deberia tener mejor accuracy que hands + pose